#### let's what we've got here...

In [1]:
import pandas as pd
import numpy as np
import itertools
import os

In [3]:
demo = pd.read_csv('Demographics.csv')
bmx = pd.read_csv('BodyMeasures.csv')
ocq = pd.read_csv('Occupation.csv')

In [6]:
print(demo.shape, bmx.shape, ocq.shape)

(9760, 144) (9278, 38) (7216, 36)


In [7]:
# no errors here coz values are the same as it should be
np.testing.assert_array_equal(demo.shape, (9760, 144))
np.testing.assert_array_equal(bmx.shape, (9278, 38))
np.testing.assert_array_equal(ocq.shape, (7216, 36))


In [8]:
# dataset would have fewer rows coz inner join drops all participants with missing physical and occupaton survey
dataset = pd.merge(demo, bmx, on='SEQN', how='inner')
dataset = pd.merge(dataset, ocq, on='SEQN', how='inner')


In [19]:
# checking if sum of all columns are equal to columns of our dataset
# -2 means we're substracting the repeated and joined SEQN colum
total_cols = demo.shape[1] + bmx.shape[1] + ocq.shape[1] - 2

In [22]:
print(dataset.shape[1], total_cols)


216 216


In [18]:
np.testing.assert_equal(dataset.shape[1], total_cols)

In [23]:
# check if new dataset has fewer rows than original demographic dataset
np.testing.assert_array_less(dataset.shape[0], demo.shape[0])

In [29]:
# remove all white spaces at once rather than specifying individual columns
# we need try/except to save ourself from an error from calling 'strip' method on numeric
for col in demo.columns:
    try:
        demo.loc[:, col] = demo.loc[:, col].str.strip()
    except AttributeError:
        pass

#### speed up checks for valid data through automation
* but we still need to choose the acceptable values ourselves
*  and then can automate the checking and replacing
* one of the options is to create a dict with 'columns' as keys and entryes as values
* for categorical data use DataFrames 'is in' method

In [31]:
valid_codes = {
    'DMDBORN': [1, 2, 3, 7, 9],
    'DMDCITZN': [1, 2, 7, 9],
}

In [37]:
for col in valid_codes.keys():
    good_indexes = demo[col].isin(valid_codes[col])
    demo.loc[~good_indexes, col] = np.nan
    # print(demo.loc[~good_indexes, col])

In [43]:
# for continuous values we can use similar methods
# but specify a valid range instead of individual values
valid_range = {
    'BMXWT': [0, 635],
    'BMXHT': [81.8, 201.3],
}

In [51]:
for col in valid_range.keys():
    good_indexes = (bmx[col] >= valid_range[col][0]) & (bmx[col] <= valid_range[col][1])
    bmx.loc[~good_indexes, col] = np.nan

#### automatically find any rows or columns with too much missing data
#### set max treshold percent to list all columns with more than 30% of the data missing


In [56]:
max_missing_perc = 30
valid_entries = demo.count()
total_rows = len(demo)
missing_percentage = (total_rows - valid_entries) / total_rows * 100


In [57]:
missing_bool = missing_percentage > max_missing_perc
print(demo.columns[missing_bool])

Index(['DMQMILIT', 'DMDBORN', 'DMDCITZN', 'DMDYRSUS', 'DMDEDUC3', 'DMDEDUC2',
       'DMDSCHOL', 'DMDMARTL', 'RIDEXPRG', 'RIDPREG', 'DMDHSEDU', 'DMAETHN',
       'DMARACE'],
      dtype='object')
